## 문서 벡터 저장소, Vector Stores  
벡터 DB는 관계형 데이터베이스가 아닌, 비정형 데이터를 저장하는데 특화된 DB다.    
RDB가 문자와 숫자 정보를 특성에 따라 열과 행의 테이블 형태로 저장되나 비정형 데이터는 데이터마다 의미가 완전 달라 테이블 형태로 적재되지 않는다.   
따라서 데이터 조회 방식도 달라서 쿼리를 기반으로 정형 데이터의 정확한 행과 열을 추출하여 정보 조회하는것과 달리 벡터 정보들 중 유사도를 기반으로 가장 비슷한 것을 찾는다.    
ANN(Approximate Nearest Neighbor) 알고리즘을 바탕으로 임베딩 벡터 간 유사도를 계산한다.    
    - HNSW(Hierarchical Navigable Small World : 계층적 탐색 구조), SPTAG(Space Partition Tree And Graph : 트리+그래프 탐색 구조) 방식    
벡터 DB 종류
    - 순수 벡터 DB : 무료 오픈 소스 Chroma, Weaviate, Qdrant 등, 유료 클로즈드 소스 Pinecone, ziliz / 임베딩 벡터 인덱싱 기술에 최적화 / SQL기반 기존 DB와 결합 어려움 / CRUD가 부실하여 유지보수 어려움    
    - 텍스트 전용 DB : Elasticsearch, Opensearch, Apache Lucene, Solr / 다국어 검색 지원, 커스터마이징 가능 토크나이저, 불용어 목록, n-gram 등 기능이 많고 검색 소프트웨어와 결합됨 / 유사도 계산 최적화 안됨 / 벡터 검색을 위한 추가적인 모듈 결합 필요    
    - 벡터 라이브러리 : FAISS(Facebook AI Similarity Search), Annoy, Hnswlib / 벡터 유사도 계산을 위한 라이브러리 / DB로의 기능은 부실한편    
    - 벡터 기능 추가된 NoSQL : MongoDB, neo4j, redis / 비정형, 반정형 데이터의 저장과 검색을 용이하게 설계 / 유사도 계산 추가    
    - 벡터 저장 및 검색 가능한 SQL DB : PostgreDB, Clickhouse, Kinetica / SQL쿼리 기반으로 데이터 조회 가능 / 벡터 저장 및 데이터 조회 최적화 안됨 / 벡터 검색을 위한 ANN 알고리즘 최적화 필요   
Chroma DB
    - RAG 구축 시 가장 많이 활용되는 오픈 소스 벡터 DB
    - 임베딩, 메타데이터 저장, 문서 및 질문 임베딩, 임베딩 검색 가능   
    - 장점 : 다른 DB 대비 단순한 사용성, 유사도 순위 제공, 빠른 속도      

In [ ]:
# Chroma DB 문서 저장 및 유사 문서 검색 practice

import os 
from langchain.document_loaders import PyPDFLoader 
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

api_key = os.environ["GEMINI_API_KEY"]

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)

loader = PyPDFLoader("./data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

# db = Chroma.from_documents(docs, embedding_model, persist_directory="./chroma_db")

# ChromaDB 초기화
db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

# batch_size를 줄여서 데이터 추가 (메모리 절약)
batch_size = 5  
for i in range(0, len(docs), batch_size):
    db.add_documents(docs[i:i + batch_size])
    print(f"{i + batch_size}개 문서 처리 완료") 

C:\Users\gram\AppData\Local\Temp\ipykernel_12208\3545496155.py:22: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)


5개 문서 처리 완료
10개 문서 처리 완료
15개 문서 처리 완료
20개 문서 처리 완료
25개 문서 처리 완료
30개 문서 처리 완료
35개 문서 처리 완료
40개 문서 처리 완료
45개 문서 처리 완료
50개 문서 처리 완료
55개 문서 처리 완료


In [1]:
import shutil
shutil.rmtree("./chroma_db", ignore_errors=True)

In [4]:
query="대통령의 임기는?"

# 유사문서 검색
docs = db.similarity_search(query)
print(docs[0])

# 유사문서 검색 및 유사도 출력
db.similarity_search_with_score(query)

page_content='제123조 ①국가는 농업 및 어업을 보호ㆍ육성하기 위하여 농ㆍ어촌종합개발과 그 지원등 필요한 계획을 수립ㆍ시행하
여야 한다.
②국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진다.
③국가는 중소기업을 보호ㆍ육성하여야 한다.
④국가는 농수산물의 수급균형과 유통구조의 개선에 노력하여 가격안정을 도모함으로써 농ㆍ어민의 이익을 보호
한다.' metadata={'page': 11, 'page_label': '12', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}


[(Document(metadata={'page': 11, 'page_label': '12', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='제123조 ①국가는 농업 및 어업을 보호ㆍ육성하기 위하여 농ㆍ어촌종합개발과 그 지원등 필요한 계획을 수립ㆍ시행하\n여야 한다.\n②국가는 지역간의 균형있는 발전을 위하여 지역경제를 육성할 의무를 진다.\n③국가는 중소기업을 보호ㆍ육성하여야 한다.\n④국가는 농수산물의 수급균형과 유통구조의 개선에 노력하여 가격안정을 도모함으로써 농ㆍ어민의 이익을 보호\n한다.'),
  0.5719609569110812),
 (Document(metadata={'page': 12, 'page_label': '13', 'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}, page_content='③대통령은 제1항의 목적을 달성하기 위하여 필요한 자문기구를 둘 수 있다.\n \n       제10장 헌법개정\n \n제128조 ①헌법개정은 국회재적의원 과반수 또는 대통령의 발의로 제안된다.\n②대통령의 임기연장 또는 중임변경을 위한 헌법개정은 그 헌법개정 제안 당시의 대통령에 대하여는 효력이 없다.\n \n제129조 제안된 헌법개정안은 대통령이 20일 이상의 기간 이를 공고하여야 한다.\n \n제130조 ①국회는 헌법개정안이 공고된 날로부터 60일 이내에 의결하여야 하며, 국회의 의결은 재적의원 3분의 2 이상\n의 찬성을 얻어야 한다.\n②헌법개정안은 국회가 의결한 후 30일 이내에 국민투표에 붙여 국회의원선거권자 과반수의 투표와 투표자 과반수\n의 찬성을 얻어야 한다.\n③헌법개정안이 제2항의 찬성을 얻은 때에는 헌법개정은 확정되며, 대통령은 즉시 이를 공포하여야 한다.\n  \n부칙 <제10호,1987. 10. 29.>'),
  0.5778240290782715),
 (Document(metadata={'page': 6,

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

loader = PyPDFLoader(r"./data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)


#HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embedding= HuggingFaceEmbeddings(
    model_name=model_name
)


#save to disk
db2 = Chroma.from_documents(docs, ko_embedding, persist_directory="./chroma_db_2")

# load from disk
db3 = Chroma(persist_directory="./chroma_db_2", embedding_function=ko_embedding)

query = "대통령의 임기는?"
result = db3.similarity_search(query)
print(result[0].page_content)

C:\Users\gram\AppData\Local\Temp\ipykernel_21104\3104654464.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  ko_embedding= HuggingFaceEmbeddings(
c:\Users\gram\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


: 

In [ ]:
# Chroma DB API를 활용한 문서 관리 
# Collection 기능 : 텍스트 임베딩을 포함하는 상위 개념 폴더로 데이터 추가, 교체, 삭제와 같은 기본적인 DB 관리 작업 수행한다

# Collection 객체 생성과 문서 저장

import chromadb 

# Collection을 저장할 경로 지정
client = chromadb.PersistentClient(path="collection_example")

# client 연결 확인 (미연결 시 출력하지 않음)
client.heartbeat()

1741140174787583000

In [ ]:
import os
from chromadb.utils import embedding_functions 
from chromadb.utils.embedding_functions import GoogleGenerativeAiEmbeddingFunction

# google 임베딩 모델 활용
api_key = os.environ["GEMINI_API_KEY"]

embedding_function = GoogleGenerativeAiEmbeddingFunction(model_name="models/embedding-001", api_key=api_key)

# HuggingFace 오픈 소스 임베딩 모델 활용 
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="jhgan/ko-sbert-nli")

collection = client.create_collection(name="korean_law", embedding_function=embedding_function)

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# 문서를 임베딩으로 변환하여 Collection에 저장 

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma 
from langchain.document_loaders import PyPDFLoader 

# load the document and split it into chunks
loader = PyPDFLoader(r"./data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

collection.add(
    ids = [str(i) for i in range(len(docs))],
    documents=[i.page_content for i in docs],
    metadatas=[i.metadata for i in docs]
)

In [17]:
# Collection 로드하기

# name에 collection 이름, embedding_function은 collection 저장 시 지정한 임베딩 모델 지정
collection = client.get_collection(name="korean_law", embedding_function=embedding_function)
collection

Collection(name=korean_law)

In [18]:
# Collection 내 문서 검색

# 1페이지에서 직업 선택의 자유와 유사한 청크 3개 검색
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page": 1},
)

{'ids': [['7', '5', '4']],
 'embeddings': None,
 'documents': [['제15조 모든 국민은 직업선택의 자유를 가진다.\n \n제16조 모든 국민은 주거의 자유를 침해받지 아니한다. 주거에 대한 압수나 수색을 할 때에는 검사의 신청에 의하여 법\n관이 발부한 영장을 제시하여야 한다.\n \n제17조 모든 국민은 사생활의 비밀과 자유를 침해받지 아니한다.\n \n제18조 모든 국민은 통신의 비밀을 침해받지 아니한다.\n \n제19조 모든 국민은 양심의 자유를 가진다.\n \n제20조 ①모든 국민은 종교의 자유를 가진다.\n②국교는 인정되지 아니하며, 종교와 정치는 분리된다.',
   '제12조 ①모든 국민은 신체의 자유를 가진다. 누구든지 법률에 의하지 아니하고는 체포ㆍ구속ㆍ압수ㆍ수색 또는 심문\n을 받지 아니하며, 법률과 적법한 절차에 의하지 아니하고는 처벌ㆍ보안처분 또는 강제노역을 받지 아니한다.\n②모든 국민은 고문을 받지 아니하며, 형사상 자기에게 불리한 진술을 강요당하지 아니한다.\n③체포ㆍ구속ㆍ압수 또는 수색을 할 때에는 적법한 절차에 따라 검사의 신청에 의하여 법관이 발부한 영장을 제시\n하여야 한다. 다만, 현행범인인 경우와 장기 3년 이상의 형에 해당하는 죄를 범하고 도피 또는 증거인멸의 염려가\n있을 때에는 사후에 영장을 청구할 수 있다.\n④누구든지 체포 또는 구속을 당한 때에는 즉시 변호인의 조력을 받을 권리를 가진다. 다만, 형사피고인이 스스로\n변호인을 구할 수 없을 때에는 법률이 정하는 바에 의하여 국가가 변호인을 붙인다.\n⑤누구든지 체포 또는 구속의 이유와 변호인의 조력을 받을 권리가 있음을 고지받지 아니하고는 체포 또는 구속을',
   '법제처                                                            2                                                       국가법령정보센터\n대한민국헌법\n \n 

In [ ]:
# 조건 부 문서 검색 
# 메타 데이터 기반으로 검색

# 5페이지 이후의 청크 중에서 직업 선택의 자유와 관련한 문서 3개 검색
# $eq - 일치 (string, int, float)
# $ne - 불일치 (string, int, float)
# $gt - 초과 (int, float)
# $gte - 이상 (int, float)
# $lt - 미만 (int, float)
# $lte - 이하 (int, float)
collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3,
    where={"page": {"$gte": 5}}
)

{'ids': [['47', '45', '24']],
 'embeddings': None,
 'documents': [['법제처                                                            13                                                       국가법령정보센터\n대한민국헌법\n⑤국가는 농ㆍ어민과 중소기업의 자조조직을 육성하여야 하며, 그 자율적 활동과 발전을 보장한다.\n \n제124조 국가는 건전한 소비행위를 계도하고 생산품의 품질향상을 촉구하기 위한 소비자보호운동을 법률이 정하는 바\n에 의하여 보장한다.\n \n제125조 국가는 대외무역을 육성하며, 이를 규제ㆍ조정할 수 있다.\n \n제126조 국방상 또는 국민경제상 긴절한 필요로 인하여 법률이 정하는 경우를 제외하고는, 사영기업을 국유 또는 공유\n로 이전하거나 그 경영을 통제 또는 관리할 수 없다.\n \n제127조 ①국가는 과학기술의 혁신과 정보 및 인력의 개발을 통하여 국민경제의 발전에 노력하여야 한다.\n②국가는 국가표준제도를 확립한다.',
   '지하며, 경제주체간의 조화를 통한 경제의 민주화를 위하여 경제에 관한 규제와 조정을 할 수 있다.\n \n제120조 ①광물 기타 중요한 지하자원ㆍ수산자원ㆍ수력과 경제상 이용할 수 있는 자연력은 법률이 정하는 바에 의하\n여 일정한 기간 그 채취ㆍ개발 또는 이용을 특허할 수 있다.\n②국토와 자원은 국가의 보호를 받으며, 국가는 그 균형있는 개발과 이용을 위하여 필요한 계획을 수립한다.\n \n제121조 ①국가는 농지에 관하여 경자유전의 원칙이 달성될 수 있도록 노력하여야 하며, 농지의 소작제도는 금지된다.\n②농업생산성의 제고와 농지의 합리적인 이용을 위하거나 불가피한 사정으로 발생하는 농지의 임대차와 위탁경영\n은 법률이 정하는 바에 의하여 인정된다.\n \n제122조 국가는 국민 모두의 생산 및 생활의 기반이 되는 국토의 효율적이고 균형있는 이용ㆍ개발과 보전

In [20]:
# 메타 데이터가 아닌, 문서의 키워드 포함 여부를 통한 필터링 검색

collection.query(
    query_texts=["직업 선택의 자유"],
    n_results=3, 
    where={"page": 1},
    where_document={"$contains":"직업"}
)

{'ids': [['7']],
 'embeddings': None,
 'documents': [['제15조 모든 국민은 직업선택의 자유를 가진다.\n \n제16조 모든 국민은 주거의 자유를 침해받지 아니한다. 주거에 대한 압수나 수색을 할 때에는 검사의 신청에 의하여 법\n관이 발부한 영장을 제시하여야 한다.\n \n제17조 모든 국민은 사생활의 비밀과 자유를 침해받지 아니한다.\n \n제18조 모든 국민은 통신의 비밀을 침해받지 아니한다.\n \n제19조 모든 국민은 양심의 자유를 가진다.\n \n제20조 ①모든 국민은 종교의 자유를 가진다.\n②국교는 인정되지 아니하며, 종교와 정치는 분리된다.']],
 'uris': None,
 'data': None,
 'metadatas': [[{'page': 1,
    'page_label': '2',
    'source': './data/대한민국헌법(헌법)(제00010호)(19880225).pdf'}]],
 'distances': [[395.47450447143063]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [1]:
# import chromadb

# # Chroma DB 클라이언트 생성 (데이터베이스 연결)
# client = chromadb.Client()

# # Collection 생성 (테이블 생성)
# collection1 = client.create_collection(name="my_collection_1")
# collection2 = client.create_collection(name="my_collection_2")

# # Collection 1에 데이터 추가
# collection1.add(
#     embeddings=[[1.1, 2.3, 3.2], [4.5, 6.7, 8.9]],
#     documents=["This is document 1", "This is document 2"],
#     ids=["id1", "id2"]
# )

# # Collection 2에 데이터 추가
# collection2.add(
#     embeddings=[[9.0, 8.7, 6.5], [4.3, 2.1, 1.0]],
#     documents=["This is document 3", "This is document 4"],
#     ids=["id3", "id4"]
# )

# # Collection 1 에서 검색
# results_collection1 = collection1.query(
#     query_embeddings=[[3.2, 4.3, 5.4]],
#     n_results=2
# )
# print("Collection 1 검색 결과:", results_collection1)

# # Collection 2 에서 검색
# results_collection2 = collection2.query(
#     query_embeddings=[[0.9, 1.2, 2.3]],
#     n_results=2
# )
# print("Collection 2 검색 결과:", results_collection2)